# VTK pipeline
## Rafael J. Mantilla - 201124446

We synthetise the whole pipeline to generate an imgage in the following script:

In [13]:
import vtk

gridReader = vtk.vtkRectilinearGridReader()
gridReader.SetFileName("./data/jet4_0.500.vtk")
gridReader.Update()

gridOutline = vtk.vtkRectilinearGridOutlineFilter()
gridOutline.SetInputData(gridReader.GetOutput())

plane = vtk.vtkRectilinearGridGeometryFilter()
plane.SetInputData(gridReader.GetOutput())
plane.SetExtent(0, 128, 128, 128, 0, 128)

outlineMapper = vtk.vtkPolyDataMapper()
outlineMapper.SetInputConnection(gridOutline.GetOutputPort())

geomMapper = vtk.vtkPolyDataMapper()
geomMapper.SetInputConnection(plane.GetOutputPort())

outlineActor = vtk.vtkActor()
outlineActor.SetMapper(outlineMapper)
outlineActor.GetProperty().SetColor(0, 0, 0)

gridGeomActor = vtk.vtkActor()
gridGeomActor.SetMapper(geomMapper)
gridGeomActor.GetProperty().SetRepresentationToWireframe()
gridGeomActor.GetProperty().SetColor(0.5, 0.5, 0)

calcFilter = vtk.vtkArrayCalculator()
calcFilter.SetInputConnection(gridReader.GetOutputPort())
calcFilter.AddVectorArrayName('vectors')
calcFilter.SetResultArrayName('magnitude')
calcFilter.SetFunction("mag(vectors)")
calcFilter.Update()

points = vtk.vtkPoints()
grid = calcFilter.GetOutput()
grid.GetPoints(points)
scalars = grid.GetPointData().GetArray('magnitude')

ugrid = vtk.vtkUnstructuredGrid()
ugrid.SetPoints(points)
ugrid.GetPointData().SetScalars(scalars)

for i in range (0, grid.GetNumberOfCells()):
    cell = grid.GetCell(i)
    ugrid.InsertNextCell(cell.GetCellType(), cell.GetPointIds())
    
subset = vtk.vtkMaskPoints()
subset.RandomModeOn()
subset.SetInputData(ugrid)

pointsGlyph = vtk.vtkVertexGlyphFilter()
pointsGlyph.SetInputConnection(subset.GetOutputPort())
pointsGlyph.SetInputData(ugrid)
pointsGlyph.Update()

pointsMapper = vtk.vtkPolyDataMapper()
pointsMapper.SetInputConnection(pointsGlyph.GetOutputPort())
pointsMapper.SetScalarModeToUsePointData()

pointsActor = vtk.vtkActor()
pointsActor.SetMapper(pointsMapper)

scalarRange = ugrid.GetPointData().GetScalars().GetRange()
lut = vtk.vtkLookupTable()
lut.SetNumberOfColors(200)
lut.SetHueRange(0.6, 0.4)
lut.SetVectorModeToMagnitude()
lut.Build()
isoFilter = vtk.vtkContourFilter()
isoFilter.SetInputData(ugrid)
isoFilter.GenerateValues(40, scalarRange)

isoMapper = vtk.vtkPolyDataMapper()
isoMapper.SetInputConnection(isoFilter.GetOutputPort())
isoMapper.SetLookupTable(lut)

isoActor = vtk.vtkActor()
isoActor.SetMapper(isoMapper)
isoActor.GetProperty().SetOpacity(0.2)

subset = vtk.vtkMaskPoints()
subset.RandomModeOn()
subset.SetInputConnection(gridReader.GetOutputPort())

lut = vtk.vtkLookupTable()
lut.SetNumberOfColors(256)
lut.SetHueRange(0.6, 0.4)
lut.SetVectorModeToMagnitude()
lut.Build()

hh = vtk.vtkHedgeHog()
hh.SetInputConnection(subset.GetOutputPort())
hh.SetScaleFactor(0.001)

hhm = vtk.vtkPolyDataMapper()
hhm.SetInputConnection(hh.GetOutputPort())
hhm.SetLookupTable(lut)
hhm.SetScalarVisibility(True)
hhm.SetScalarModeToUsePointFieldData()
hhm.SelectColorArray('vectors')
hhm.SetScalarRange((gridReader.GetOutput().GetPointData().GetVectors().GetRange(-1)))

hedgeActor = vtk.vtkActor()
hedgeActor.SetMapper(hhm)

renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

#------- Un-Comment any actor you want to visualize
renderer.AddActor(gridGeomActor)
renderer.AddActor(outlineActor)
renderer.AddActor(hedgeActor)
renderer.AddActor(isoActor)
#renderer.AddActor(pointsActor)

renderer.SetBackground(1, 1, 1)
 
renWin.SetSize(800, 800)
 
# interact with data
renWin.Render()
iren.Start()
